# Fix api key not updated in bullet achievement

In [7]:
import os
from dotenv import load_dotenv
import google.generativeai as genai

# Clear any existing environment variable
os.environ.pop("GEMINI_API_KEY", None)

# Load environment variables from .env file
load_dotenv()

# Retrieve the API key from environment variables
gemini_api_key = os.getenv("GEMINI_API_KEY")

genai.configure(api_key=gemini_api_key)

safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
]

model = genai.GenerativeModel(
    model_name="gemini-1.5-pro", safety_settings=safety_settings
)

In [8]:
import google.generativeai as genai
import re


def cleanupAgent(userInput):
    prompt = f"""
    High-level instruction:
    1. Remove filler words and phrases (e.g., "um", "like", "you know")
    2. Correct grammatical errors
    3. Improve sentence structure for clarity
    4. Maintain the original meaning and tone of the input
    5. Do not add any new information

    Example input:
    "So, um, I'm like working on this really cool project, you know? It's kinda about using AI to help people tell their stories better and stuff."

    Example output:
    "I'm working on an exciting project that uses AI to help people tell their stories more effectively."

    Now, please clean up the following user input:
    {userInput}
    """

    response = model.generate_content(prompt)
    return response.text


def translateAgent(cleanedInput):
    prompt = f"""
    High-level instruction:
    1. Ensure consistent terminology throughout the text
    2. Replace jargon or technical terms with more accessible language
    3. Maintain the original meaning and intent of the input
    4. Adjust language to be suitable for a general audience
    5. Preserve any key technical terms that are essential to the story

    Example input:
    "Our novel machine learning algorithm leverages neural networks to optimize natural language processing tasks, resulting in a 37% improvement in sentiment analysis accuracy."

    Example output:
    "Our innovative AI system uses advanced computing techniques to better understand human language, leading to a significant 37% improvement in accurately determining the emotions behind text."

    Now, please translate the following cleaned input:
    {cleanedInput}
    """

    response = model.generate_content(prompt)
    return response.text


def regexExtraction(translatedInput):
    prompt = f"""
    High-level instruction:
    1. Extract key details from the input text using regular expressions
    2. Identify and extract the following information:
       - Name
       - Field of study or work
       - Main problem addressed
       - Proposed solution
       - Key achievement or impact
       - Future goal
    3. Format the extracted information as a JSON object
    4. If any information is not found, use "N/A" as the value

    Example input:
    "Jane Doe, an environmental engineer, is tackling ocean plastic pollution. She developed a biodegradable plastic alternative made from algae, which has reduced plastic waste by 50% in a pilot city. Her goal is to achieve global adoption of this eco-friendly packaging."

    Example output:
    {{
        "name": "Jane Doe",
        "field": "Environmental Engineering",
        "problem": "Ocean plastic pollution",
        "solution": "Biodegradable plastic alternative made from algae",
        "impact": "50% reduction in plastic waste in pilot city",
        "future_goal": "Global adoption of eco-friendly packaging"
    }}

    Now, please extract key details from the following translated input:
    {translatedInput}
    """

    response = model.generate_content(prompt)
    return response.text


def createPitch(extractedDetails, duration):
    prompt = f"""
    High-level instruction:
    1. Create a compelling elevator pitch using the extracted details
    2. Ensure the pitch fits within the specified duration ({duration} seconds)
    3. Include a strong opening hook
    4. Clearly state the problem and your unique solution
    5. Highlight key achievements and impact
    6. End with a clear call to action
    7. Adjust the level of detail based on the duration (more concise for shorter durations)
    8. Ensure the pitch flows naturally and engagingly

    Example input (extracted details):
    {{
        "name": "Jane Doe",
        "field": "Environmental Engineering",
        "problem": "Plastic pollution in oceans",
        "solution": "Biodegradable algae-based plastic alternative",
        "impact": "50% reduction in plastic waste in pilot city",
        "future_goal": "Global adoption of eco-friendly packaging"
    }}

    Example output (30-second pitch):
    "Hi, I'm Jane Doe, an environmental engineer tackling ocean plastic pollution. Our oceans are choking on plastic, but I've developed a game-changing solution: a biodegradable plastic alternative made from algae. In our pilot city, we've already cut plastic waste by 50%. Imagine a world where packaging nurtures our planet instead of polluting it. With your support, we can make this a global reality. Join us in revolutionizing packaging and saving our oceans, one algae wrapper at a time."

    Now, please create a {duration}-second pitch using the following extracted details:
    {extractedDetails}
    """

    response = model.generate_content(prompt)
    return response.text


def humanizeAgent(shortPitch, mediumPitch, longPitch):
    prompt = f"""
    High-level instruction:
    1. Enhance the natural language and conversational tone of each pitch
    2. Add appropriate verbal fillers and pauses (e.g., "you see", "now", "well")
    3. Incorporate rhetorical devices (e.g., metaphors, analogies) where appropriate
    4. Ensure consistency in voice and style across all pitch lengths
    5. Maintain the core message and key points of each pitch
    6. Adjust language to sound more spontaneous and less rehearsed
    7. Add personal touches or anecdotes where appropriate

    Example input (30-second pitch):
    "Hi, I'm Jane Doe, an environmental engineer tackling ocean plastic pollution. Our oceans are choking on plastic, but I've developed a game-changing solution: a biodegradable plastic alternative made from algae. In our pilot city, we've already cut plastic waste by 50%. Imagine a world where packaging nurtures our planet instead of polluting it. With your support, we can make this a global reality. Join us in revolutionizing packaging and saving our oceans, one algae wrapper at a time."

    Example output (humanized 30-second pitch):
    "Hey there! Jane Doe here. You know how our oceans are drowning in plastic? Well, as an environmental engineer, I couldn't just stand by. So, get this – we've created this amazing plastic alternative from algae. It's like magic – it just biodegrades! In our test city, plastic waste is down by half already. Can you imagine a world where our packaging feeds the planet instead of killing it? Pretty cool, right? With a little help, we could take this global. So, what do you say? Ready to wrap the world in algae and save our oceans?"

    Now, please humanize the following pitches:
    Short Pitch (30 seconds): {shortPitch}
    Medium Pitch (60 seconds): {mediumPitch}
    Long Pitch (300 seconds): {longPitch}
    """

    response = model.generate_content(prompt)
    return response.text


def getExpertFeedback(humanizedPitches):
    prompt = f"""
    High-level instruction:
    1. Analyze the provided pitches from the perspective of four experts:
       - Storytelling Expert
       - Industry Expert
       - Presentation Expert
       - Audience Engagement Expert
    2. For each expert, provide specific feedback on the strengths and areas for improvement
    3. Offer actionable suggestions for enhancing each pitch
    4. Ensure feedback is constructive, specific, and tailored to each pitch length
    5. Highlight any inconsistencies or missed opportunities across the different pitch lengths

    Example input:
    Short Pitch (30 seconds): [30-second pitch content]
    Medium Pitch (60 seconds): [60-second pitch content]
    Long Pitch (300 seconds): [300-second pitch content]

    Example output:
    {{
        "StorytellingExpert": {{
            "strengths": ["Compelling narrative arc", "Effective use of imagery"],
            "improvements": ["Strengthen emotional connection", "Clarify central theme"],
            "suggestions": ["Incorporate a personal anecdote", "Use a recurring motif to tie the story together"]
        }},
        "IndustryExpert": {{
            "strengths": ["Accurate problem statement", "Innovative solution"],
            "improvements": ["Elaborate on technical feasibility", "Address potential industry pushback"],
            "suggestions": ["Include a brief comparison with current industry standards", "Mention any partnerships or collaborations"]
        }},
        "PresentationExpert": {{
            "strengths": ["Clear and concise language", "Effective use of pauses"],
            "improvements": ["Vary vocal pitch and tempo", "Enhance opening hook"],
            "suggestions": ["Practice emphasizing key words", "Start with a thought-provoking question"]
        }},
        "AudienceEngagementExpert": {{
            "strengths": ["Strong call to action", "Relatable examples"],
            "improvements": ["Increase audience interaction", "Tailor message to specific audience"],
            "suggestions": ["Include a rhetorical question", "Relate the problem to everyday experiences"]
        }}
    }}

    Now, please provide expert feedback on the following humanized pitches:
    {humanizedPitches}
    """

    response = model.generate_content(prompt)
    return response.text


def processUserStory(userInput):
    cleanedInput = cleanupAgent(userInput)
    translatedInput = translateAgent(cleanedInput)
    extractedDetails = regexExtraction(translatedInput)

    shortPitch = createPitch(extractedDetails, duration=30)
    mediumPitch = createPitch(extractedDetails, duration=60)
    longPitch = createPitch(extractedDetails, duration=300)

    humanizedPitches = humanizeAgent(shortPitch, mediumPitch, longPitch)
    expertFeedback = getExpertFeedback(humanizedPitches)

    return {
        "shortPitch": humanizedPitches["short"],
        "mediumPitch": humanizedPitches["medium"],
        "longPitch": humanizedPitches["long"],
        "expertFeedback": expertFeedback,
    }

In [9]:
# Example usage
if __name__ == "__main__":
    user_story = input("Please tell us about your project or accomplishment: ")
    result = processUserStory(user_story)
    print(result)

TypeError: string indices must be integers, not 'str'